In [1]:
import jax.numpy as jnp
import numpy as np
import jax
import optax
import scipy
from scipy.spatial.distance import cdist
from scipy.integrate import RK45
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import equinox as eqx
from jax.nn.initializers import variance_scaling
from jax.nn.initializers import he_normal
from tqdm import tqdm


seed = 42
np.random.seed(42)
key = jax.random.key(seed)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



W0000 00:00:1758816152.633314 47168428 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1758816152.667397 47168428 service.cc:145] XLA service 0x1412b21b0 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1758816152.667437 47168428 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1758816152.670065 47168428 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1758816152.670089 47168428 mps_client.cc:384] XLA backend will use up to 11452858368 bytes on device 0 for SimpleAllocator.


In [2]:
dataset = np.load('/Users/haydenoutlaw/Documents/Research/opnet/deep-operator-networks/data/burgers_dataset.npz', allow_pickle = True)

list(dataset.keys())

t_grid = dataset['t']
x_grid = dataset['x']

data = dataset['samples']
n_samp = data.shape[0]

In [3]:
u = np.array([i['params'] for i in data])
s = np.array([i['solution'] for i in data])

In [4]:
train_indices, test_indices = train_test_split(np.arange(n_samp), test_size = 0.33, random_state = seed)
u_train, u_test = u[train_indices], u[test_indices]
s_train, s_test = s[train_indices], s[test_indices]

In [5]:
s_train.shape

(295, 201, 201)

In [14]:
def get_point_batch(batch_size, params, s, x, t, key):
    """
    Build a batch of random samples, each as [param, x, t, s].
    
    params: (N, 2) array of parameters
    s:      (N, nt, nx) array of solutions
    x:      (nx,) spatial grid
    t:      (nt,) time grid
    key:    jax.random.PRNGKey
    
    Returns:
        samples: list of length batch_size
                 each sample = [param (2,), x (scalar), t (scalar), s (scalar)]
        new_key: updated PRNG key
    """
    N, nt, nx = s.shape
    
    # Randomly choose indices
    key, subkey1, subkey2, subkey3 = jax.random.split(key, 4)
    sample_idx = jax.random.randint(subkey1, (batch_size,), 0, N)
    t_idx      = jax.random.randint(subkey2, (batch_size,), 0, nt)
    x_idx      = jax.random.randint(subkey3, (batch_size,), 0, nx)
    
    # Build per-sample list
    samples = []
    for i in range(batch_size):
        p = params[sample_idx[i]]
        xi = x[x_idx[i]]
        ti = t[t_idx[i]]
        si = s[sample_idx[i], t_idx[i], x_idx[i]]
        samples.append([p, xi, ti, si])
    
    return samples, key


In [18]:
samp, key = get_point_batch(10, u_train, s_train, t_grid, x_grid, key)
samp[0]

[array([0.77, 0.99], dtype=float32),
 np.float32(0.28),
 np.float32(-1.35),
 np.float32(0.06577639)]